In [3]:
import pandas as pd

# csv
# samsung_df = pd.read_csv(f'csv/samsung_data_set.csv',encoding='cp949')
# kakao_df = pd.read_csv(f'csv/카카오 뉴스 제목8.csv',encoding='cp949')
# sk_df = pd.read_csv(f'csv/sk_data_label.csv',encoding='cp949')
# hyundai_df = pd.read_csv(f'csv/hyundai_data_label.csv',encoding='cp949')

# samsung_df.dropna(inplace=True)
# kakao_df.dropna(inplace=True)
# sk_df.dropna(inplace=True)
# hyundai_df.dropna(inplace=True)

new_kakao_df = pd.read_csv(f'csv/카카오 뉴스 제목 부정.csv',encoding='cp949')
tot_df = pd.read_csv(f'csv/self_labeled_data.csv')

In [43]:
# total_df = pd.concat([samsung_df,kakao_df,sk_df,hyundai_df])
# total_df = total_df.reset_index()
# total_df = total_df.drop('index',axis=1)

In [4]:
new_kakao_df

,date,title,self_label
0,2021-09-15 20:12,"[단독] 김범수 동생, 카카오 '옥상옥 지주사'서 퇴직금만 14억",1
1,2021-09-15 6:01,카카오 세상' 꿈꿨지만…택시 90% 완전 장악이 독 됐다,0
2,2021-09-15 19:52,"[단독]국회 정무위, 카카오 김범수 '공정위 국감' 증인대 세운다",0
3,2021-09-15 9:29,"카카오엔터테인먼트, 경력 개발자 첫 공개채용…최대 세 자릿수 규모",1
4,2021-09-15 7:27,카카오 김범수 공정위 조사…“고의성 입증시 검찰 고발”,0
...,...,...,...
94,2021-09-15 13:00,전방위 압박에 카카오 상생방안 발표...규제 피할까?,0
95,2021-09-15 11:53,착잡·당혹…뒤숭숭한 카카오 직원들,0
96,2021-09-15 11:45,"네이버·카카오페이, '소상공인 수수료' 카드사보다 최대 3배 높아",0
97,2021-09-15 3:02,"백기 든 카카오, 골목상권 손뗀다",0


In [2]:
total_df.to_csv(f'csv/total_df.csv', index=False)

NameError: name 'total_df' is not defined

In [3]:
total_df = pd.read_csv('csv/total_df.csv',encoding='cp949')

In [4]:
total_df

,date,title,self_label
0,2023-05-30 9:00,"삼성전자, 갤럭시·노트북·TV서 '자가 수리' 국내 도입",1
1,2023-05-30 10:43,"삼성전자 갤럭시, 이제 내손으로 고친다",1
2,2023-05-30 11:09,"[특징주] 삼성전자·SK하이닉스, 52주 신고가 기록",1
3,2023-05-30 14:01,"""청년에 일경험""…SK하이닉스·삼성전자 등 2100명 모집",1
4,2023-05-30 11:21,작년 주주 배당액 1위는 삼성전자…총 배당금은 6.7%↓,0
...,...,...,...
552,2023-06-05 18:28,"울산 현대, 임금 인상·고용 안정 놓고 올 임단협도 험로 예고",0
553,2023-06-05 16:30,"현대차·이케아코리아, 광명점서 친환경차 전시",1
554,2023-06-06 12:37,‘커넥티드카’ 기술경쟁 심화…5G-V2X 한국이 선도,1
555,2023-06-06 10:25,"""전기차로 전국일주"" 싱가포르 관광객 제주 입도",1


In [ ]:
# 남은 할일 
# 기사 제목 끌고 와서 라벨링 해준다
# 그 데이터 가지고 정확도를 예측한다. 이 때 사용될 건 accuracy, f1 score 

import pandas as pd
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


# csv
df = pd.read_csv(f'csv/total_df.csv',encoding="cp949")
df = new_kakao_df
# df = pd.read_csv(f'csv/kakao_data_label.csv',encoding='cp949') # encoding='cp949'
# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# df['content'] = df['title']

# title 열에 대해 형태소 분석 적용

df['title'] = df['title'].astype(str).apply(tokenize)
# for i in range(df.shape[0]):
#     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#df.to_csv(f'csv/labeled_total_df.csv', index=False)
df.to_csv(f'csv/labeled_카카오.csv',index=False)

In [70]:
# df.to_csv(f'csv/test_06060840.csv', index=False)

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np

# X_train = np.expand_dims(samsung_train['sentiment'].values,axis=1)
# X_test =np.expand_dims(samsung_test['sentiment'].values,axis=1)
# y_train =np.expand_dims(samsung_train['label'].values,axis=1)
# y_test = np.expand_dims(samsung_test['label'].values,axis=1)


X = np.expand_dims(df['sentiment'].values,axis=1)
y = np.expand_dims(df['self_label'].values,axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=156)


In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train,y_train)
dt_pred = dt_clf.predict(X_test)
print(f'DecisionTreeClassifier 정확도: {accuracy_score(y_test,dt_pred)}')

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train,y_train)
rf_pred = rf_clf.predict(X_test)
print(f"RandomForestClassifier 정확도: {accuracy_score(y_test,rf_pred)}")

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)
print(f"LogisticRegression 정확도: {accuracy_score(y_test,lr_pred)}")

DecisionTreeClassifier 정확도: 0.8571428571428571
RandomForestClassifier 정확도: 0.8660714285714286
LogisticRegression 정확도: 0.8660714285714286


C:\Users\ahn\AppData\Local\Temp\ipykernel_14908\993524692.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train,y_train)
C:\Users\ahn\anaconda3\envs\CapstonStockProject\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
print(dt_pred)
print(rf_pred)
print(lr_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]


In [12]:
count_1 = 0
count_0 = 0
for i in dt_pred:
    if i ==1:
        count_1 +=1
    else:
        count_0 +=1

if count_1 >= count_0:
    print("상승을 에측")
else:
    print("하락을 예측")


상승을 에측


In [15]:
# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
df = pd.read_csv(f'csv/삼성전자_test_data_9.csv')

# DecisionTreeClassifier 학습/예측/평가
dt_pred = dt_clf.predict(np.expand_dims(df['sentiment'].values,axis=1))


# RandomForestClassifier 학습/예측/평가
rf_pred = rf_clf.predict(np.expand_dims(df['sentiment'].values,axis=1))


# LogisticRegression 학습/예측/평가
lr_pred = lr_clf.predict(np.expand_dims(df['sentiment'].values,axis=1))


In [16]:
count_1 = 0
count_0 = 0
for i in dt_pred:
    if i ==1:
        count_1 +=1
    else:
        count_0 +=1

if count_1 >= count_0:
    print("상승을 에측")
else:
    print("하락을 예측")

상승을 에측


In [17]:
print(dt_pred)
print(rf_pred)
print(lr_pred)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
